#### Download ECCC weather station data from https://dd.weather.gc.ca

In [ ]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path

def download_csv_files(url, folder_path):
    # Local directory where files will be saved using pathlib
    save_dir = Path(folder_path)
    save_dir.mkdir(parents=True, exist_ok=True) 

    # Get the webpage content
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to connect to {url}")
        return

    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the CSV file links
    for link in soup.find_all('a'):
        file_name = link.get('href')
        if file_name.endswith('.csv'):
            file_url = url + file_name

            # Download the CSV file
            print(f"Downloading {file_name}...")
            csv_response = requests.get(file_url)

            # Save the CSV file using pathlib
            file_path = save_dir / file_name
            file_path.write_bytes(csv_response.content)

    print("All files downloaded!")

# Example usage
url = "https://dd.weather.gc.ca/climate/observations/hourly/csv/AB/"
download_folder = r"C:\Users\Sunbeam\Downloads\csv_files"
download_csv_files(url, download_folder)

#### Filter and analyze station data and create a merged csv

In [2]:
import arcpy
import re
import pandas as pd
from pathlib import Path

arcpy.env.overwriteOutput = True

def filter_eccc_station_data(csv_dir, months, shp_bbox, weather_param=None):
    """
    Filters weather data from CSV files in a directory based on specified months and spatial polygon, 
    merges the data, saves it as a combined CSV, and generates a summary per station.

    Parameters:
    csv_dir (str): Directory containing CSV files with weather data.
    months (list): List of months (as integers) to filter the data.
    shp_polygon (str): Path to the shapefile containing the polygon for spatial filtering.
    weather_param (list, optional): List of weather parameters to include in the filtered data.
    """
    
    default_columns = ['longitude (x)', 'latitude (y)', 'station name', 'climate id', 'date/time (lst)', 'year', 'month', 'day', 'time (lst)']
    weather_param = weather_param or []
    original_weather_param = [param.lower() for param in weather_param]
    cleaned_weather_param = [re.sub(r'[^a-zA-Z]', '', param.lower())[:10] for param in weather_param]
    selected_columns = default_columns + original_weather_param

    with arcpy.da.SearchCursor(shp_bbox, ["SHAPE@"]) as cursor:
        bbox_polygon = next(cursor)[0]

    csv_files = list(Path(csv_dir).glob("*.csv"))
    station_dict, lat_long_id_dict, all_data = {}, {}, []

    for csv_file in csv_files:
        try:
            weather_data = pd.read_csv(csv_file, encoding='utf-8', on_bad_lines='skip', engine='python')
        except UnicodeDecodeError:
            weather_data = pd.read_csv(csv_file, encoding='ISO-8859-1', on_bad_lines='skip', engine='python')

        weather_data.columns = weather_data.columns.str.strip().str.lower()
        if 'station name' not in weather_data.columns:
            continue

        if 'date/time (lst)' in weather_data.columns:
            weather_data['date/time (lst)'] = pd.to_datetime(weather_data['date/time (lst)'], errors='coerce')
            weather_data = weather_data[weather_data['date/time (lst)'].dt.month.isin(months)]
        if weather_data.empty:
            continue

        longitude_col = [col for col in weather_data.columns if 'longitude' in col][0]
        latitude_col = [col for col in weather_data.columns if 'latitude' in col][0]
        weather_data['station_point'] = weather_data.apply(
            lambda row: arcpy.PointGeometry(arcpy.Point(row[longitude_col], row[latitude_col]), arcpy.SpatialReference(4326)), axis=1
        )
        weather_data = weather_data[weather_data['station_point'].apply(lambda pt: pt.within(bbox_polygon))]

        if weather_data.empty:
            continue

        for _, row in weather_data.iterrows():
            lat_long_id_key = (row[longitude_col], row[latitude_col], row['climate id'])
            if lat_long_id_key not in lat_long_id_dict:
                lat_long_id_dict[lat_long_id_key] = {
                    'station_name': row['station name'],
                    'climate_id': row['climate id'],
                    'location': row['station_point']
                }
                station_dict.setdefault((row['station name'], row['climate id']), {'location': row['station_point'], 'data': pd.DataFrame()})

        all_data.append(weather_data[selected_columns])

    for (station_name, climate_id), station_info in station_dict.items():
        station_data = pd.concat([data for data in all_data if data['station name'].eq(station_name).any()])
        station_data['year'] = station_data['date/time (lst)'].dt.year
        year_counts = station_data.groupby('year').size()
        year_info = ', '.join([f"{year} ({count})" for year, count in year_counts.items()])
        print(f"{station_name} (ID: {climate_id}) - {year_info}")
    
    if lat_long_id_dict:
        shapefile_name = Path(shp_bbox).parent / f"filtered_stations_{Path(shp_bbox).stem}_{'_'.join(map(str, months))}.shp"
        point_features = [info['location'] for info in lat_long_id_dict.values()]
        arcpy.CopyFeatures_management(point_features, str(shapefile_name))
        arcpy.management.AddFields(str(shapefile_name), [["station_n", "TEXT"], ["climate_id", "TEXT"]] + [[field, "TEXT"] for field in cleaned_weather_param])

        with arcpy.da.UpdateCursor(str(shapefile_name), ["station_n", "climate_id"] + cleaned_weather_param) as cursor:
            for i, row in enumerate(cursor):
                key = list(lat_long_id_dict.keys())[i]
                row[0], row[1] = lat_long_id_dict[key]['station_name'], lat_long_id_dict[key]['climate_id']
                cursor.updateRow(row)

    # Merge all the data and save as a CSV
    merged_csv_path = Path(shp_bbox).parent / f"ECCC_station_data_{Path(shp_bbox).stem}_{'_'.join(map(str, months))}.csv"
    merged_data = pd.concat(all_data, ignore_index=True)
    merged_data.rename(columns={original: cleaned for original, cleaned in zip(original_weather_param, cleaned_weather_param)}, inplace=True)
    merged_data.to_csv(merged_csv_path, index=False)
    print(f"Merged CSV file saved at: {merged_csv_path}")

    summary = []
    for station_name, station_data in merged_data.groupby('station name'):
        null_counts = station_data[cleaned_weather_param].isnull().sum().to_dict()

        summary.append({
            'Station name': station_name,
            'First Year': station_data['date/time (lst)'].dt.year.min(),
            'Total Records': len(station_data),
            **{f"Null Values - {param}": null_counts.get(param, 0) for param in cleaned_weather_param}
        })

    # Save the summary as a CSV
    summary_df = pd.DataFrame(summary)
    summary_output_path = Path(shp_bbox).parent / f"ECCC_station_summary_{Path(shp_bbox).stem}_{'_'.join(map(str, months))}.csv"
    summary_df.to_csv(summary_output_path, index=False)
    print(f"Station summary saved at: {summary_output_path}")
    
# Example usage
csv_dir = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\ECCC_Data"
months = [i for i in range(1, 13)]
# shp_polygon = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\BBox\BBox_small.shp"
shp_bbox = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\BBox\BBox_large.shp"
weather_param = ['Temp (°C)', 'Wind Dir (10s deg)']

filter_eccc_station_data(csv_dir, months, shp_bbox, weather_param)


C:\Users\Sunbeam\AppData\Local\Temp\ipykernel_10368\1628533076.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_data['date/time (lst)'] = pd.to_datetime(weather_data['date/time (lst)'], errors='coerce')


CORONATION A (ID: 3011880) - 1953 (8760), 1954 (8760), 1955 (8760), 1956 (8784), 1957 (8760), 1958 (8760), 1959 (8760), 1960 (8784), 1961 (8760), 1962 (8760), 1963 (8760), 1964 (8784), 1965 (8760), 1966 (8760), 1967 (8760), 1968 (8784), 1969 (8760), 1970 (8760), 1971 (8760), 1972 (8784), 1973 (8760), 1974 (8760), 1975 (8760), 1976 (8784), 1977 (8760), 1978 (8755), 1979 (8755), 1980 (8779), 1981 (8755), 1982 (8755), 1983 (8760), 1984 (8784), 1985 (8760), 1986 (8760), 1987 (8760), 1988 (8784), 1989 (8760), 1990 (8760), 1991 (8760), 1992 (8784), 1993 (8760), 1994 (2328)
CORONATION (AUT) (ID: 3011885) - 1994 (8016), 1995 (8760), 1996 (8784), 1997 (8760), 1998 (8760), 1999 (8760), 2000 (8784), 2001 (8760), 2002 (8760), 2003 (8400)
CORONATION CLIMATE (ID: 3011887) - 1999 (16), 2000 (30), 2001 (1508), 2002 (20), 2003 (1209), 2004 (8784), 2005 (8760), 2006 (8760), 2007 (8760), 2008 (8784), 2009 (8760), 2010 (8760), 2011 (8760), 2012 (8784), 2013 (8760), 2014 (8760), 2015 (8760), 2016 (8784), 2

#### Count the wind directions in two class - 210-280 and all the remainings and output in CSV

In [7]:
import pandas as pd
import re
from pathlib import Path

def process_eccc_station_data(input_csv, aggregation='yearly'):
    """
    Processes weather station data from a CSV file, handling both individual measurements and binned counts.
    Counts wind directions within specific groups (210-279 and 280-209) and outputs a pivoted CSV file.

    Parameters:
    - input_csv: Path to the input CSV file containing weather station data.
    - aggregation: Aggregation level, either 'yearly' or '5-yearly'.
    """
    df = pd.read_csv(input_csv, low_memory=False)

    # Detect if CSV format is binned (wind direction bins) or individual measurements
    is_binned_format = any(re.match(r'^\d+-\d+$', col) for col in df.columns)

    if is_binned_format:        
        dir_210_279_bins = [col for col in df.columns if re.match(r'^(21[0-9]|2[0-7][0-9])-2[0-7][0-9]$', col)]
        dir_other_bins = [col for col in df.columns if re.match(r'^\d+-\d+$', col) and col not in dir_210_279_bins]

        print("Columns in wind_dir_210_279 group:", dir_210_279_bins)

        df['wind_dir_210_279'] = df[dir_210_279_bins].sum(axis=1)
        df['wind_dir_280_209'] = df[dir_other_bins].sum(axis=1)
    else:
        df = df.dropna(subset=['winddirsde'])
        df['actual_wind_dir'] = df['winddirsde'] * 10
        df['wind_dir_210_279'] = df['actual_wind_dir'].between(210, 279).astype(int)
        df['wind_dir_280_209'] = (~df['actual_wind_dir'].between(210, 279)).astype(int)

    if aggregation == '5-yearly':
        df['aggregation_period'] = (df['year'] // 5 * 5).astype(str) + '-' + (df['year'] // 5 * 5 + 4).astype(str)
        pivot_table = df.pivot_table(
            values=['wind_dir_210_279', 'wind_dir_280_209'],
            index=['aggregation_period', 'month'],
            aggfunc='sum'
        ).reset_index()
    elif aggregation == 'yearly':
        df['aggregation_period'] = df['year'].astype(str)
        pivot_table = df.pivot_table(
            values=['wind_dir_210_279', 'wind_dir_280_209'],
            index=['aggregation_period', 'month'],
            aggfunc='sum'
        ).reset_index()
    else:
        raise ValueError("Invalid aggregation level. Please specify either 'yearly' or '5-yearly'.")

    # Save the results to a new CSV file
    output_csv = Path(input_csv).with_name(f"{Path(input_csv).stem}_{aggregation}_pivoted.csv")
    pivot_table.to_csv(output_csv, index=False)
    print(f"Processed data has been saved to {output_csv}")

# Example usage
input_csv_path = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\Daily_multilevel\Windir_1950_2024_Dec\wind_direction_15deg_data_stream-oper_BBox_small_masked.csv"
# input_csv_path = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\Daily_multilevel\Windir_1950_2024_Dec\wind_direction_15deg_data_stream-oper_BBox_large_masked.csv"
process_eccc_station_data(input_csv_path, aggregation='yearly')


Columns in wind_dir_210_279 group: ['210-225', '225-240', '240-255', '255-270']
Processed data has been saved to D:\UCalgary_Lectures\GEOG_683\Data_workspace\Daily_multilevel\Windir_1950_2024_Dec\wind_direction_15deg_data_stream-oper_BBox_small_masked_yearly_pivoted.csv


### Process ACIS weather station data

In [ ]:
from pathlib import Path
import pandas as pd

def process_acis_data(acis_data_path, coords_data_path):
    acis_data_path = Path(acis_data_path)
    output_dir = acis_data_path.parent
    
    # Load data
    acis_data = pd.read_csv(acis_data_path, low_memory=False)
    coords_data = pd.read_excel(coords_data_path)
    
    # Process data
    wd_at_data = acis_data[acis_data['SENSOR_CD'].isin(['WD', 'AT'])].copy()
    wd_at_data['TIMESTAMP'] = pd.to_datetime(wd_at_data['TIMESTAMP'], format='%Y-%m-%d-%H.%M.%S.%f')
    wd_at_data['READING'] = pd.to_numeric(wd_at_data['READING'], errors='coerce')
    
    wd_at_pivot = wd_at_data.pivot_table(index=['NAME', 'TIMESTAMP'], 
                                         columns='SENSOR_CD', 
                                         values='READING', 
                                         aggfunc='first').reset_index()
    wd_at_pivot.columns.name = None
    
    merged_data = pd.merge(wd_at_pivot, coords_data, left_on='NAME', right_on='Station names', how='left')[
        ['NAME', 'TIMESTAMP', 'WD', 'AT', 'X', 'Y']
    ]
    merged_data.columns = ['station name', 'datetime', 'winddir', 'atmostemp', 'longitude', 'latitude']
    
    # Generate merged CSV
    output_path = output_dir / "ACIS_station_data.csv"
    merged_data.to_csv(output_path, index=False)
    print(f"Merged data saved to '{output_path}'.")

    # Generate summary for each station
    summary = []
    for station, group in merged_data.groupby('station name'):
        first_year = group['datetime'].dt.year.min()
        total_records = len(group)
        null_counts = group[['winddir', 'atmostemp']].isnull().sum().to_dict()  # Count nulls for each parameter
        
        summary.append({
            'Station name': station,
            'First Year': first_year,
            'Total Records': total_records,
            'Null Values - winddir': null_counts.get('winddir', 0),
            'Null Values - atmostemp': null_counts.get('atmostemp', 0)
        })
    
    # Convert summary to DataFrame
    summary_df = pd.DataFrame(summary)
    
    # Generate summary CSV
    summary_output_path = output_dir / "ACIS_station_summary.csv"
    summary_df.to_csv(summary_output_path, index=False)
    print(f"Station summary saved to '{summary_output_path}'.")

# Example usage
acis_data_path = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\ACIS_Data\select stn hrly data dec-feb at wd.csv"
coords_data_path =  r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\ACIS_Data\ACIS_Station_names.xlsx"
process_acis_data(acis_data_path, coords_data_path)


Merged data saved to 'D:\UCalgary_Lectures\GEOG_683\Data_workspace\ACIS_Data\ACIS_station_data.csv'.
Station summary saved to 'D:\UCalgary_Lectures\GEOG_683\Data_workspace\ACIS_Data\ACIS_station_summary.csv'.


In [ ]:
import arcpy
import pandas as pd
from pathlib import Path

arcpy.env.overwriteOutput = True

def filter_acis_station_data(csv_path, months, shp_polygon, weather_param=None):
    """
    Filters ACIS weather station data based on specified months and a spatial polygon,
    and saves the filtered data as a CSV.

    Parameters:
    csv_path (str): Path to the ACIS merged CSV file.
    months (list): List of months (as integers) to filter the data.
    shp_polygon (str): Path to the shapefile containing the polygon for spatial filtering.
    weather_param (list, optional): List of weather parameters to include in the filtered data. Defaults to None.
    """
    
    default_columns = ['longitude', 'latitude', 'station name', 'datetime']
    weather_param = weather_param or ['winddir', 'atmostemp']
    
    # Load data
    acis_data = pd.read_csv(csv_path, encoding='utf-8', on_bad_lines='skip', engine='python')
    acis_data.columns = acis_data.columns.str.strip().str.lower()

    # Check that all columns are in the DataFrame
    selected_columns = default_columns + [col for col in weather_param if col in acis_data.columns]

    # Filter by specified months
    acis_data['datetime'] = pd.to_datetime(acis_data['datetime'], errors='coerce')
    acis_data = acis_data[acis_data['datetime'].dt.month.isin(months)]
    if acis_data.empty:
        print("No data available for the specified months.")
        return

    # Spatial filtering
    with arcpy.da.SearchCursor(shp_polygon, ["SHAPE@"]) as cursor:
        bbox_polygon = next(cursor)[0]

    acis_data['station_point'] = acis_data.apply(
        lambda row: arcpy.PointGeometry(arcpy.Point(row['longitude'], row['latitude']), arcpy.SpatialReference(4326)), axis=1
    )
    acis_data = acis_data[acis_data['station_point'].apply(lambda pt: pt.within(bbox_polygon))]

    if acis_data.empty:
        print("No data found within the specified polygon.")
        return

    # Count unique stations within the bounding box
    unique_stations_count = acis_data['station name'].nunique()
    print(f"Number of stations within the bounding box: {unique_stations_count}")

    # Select only the columns that exist in the data
    acis_data = acis_data[selected_columns]
    
    # Output filtered data as CSV
    output_path = Path(shp_polygon).parent / f"acis_filtered_acis_data_{Path(shp_polygon).stem}_{'_'.join(map(str, months))}.csv"
    acis_data.to_csv(output_path, index=False)
    print(f"Filtered ACIS data saved at: {output_path}")

# Example usage
csv_path = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\ACIS_Data\merged_station_data.csv"
months = [1]
shp_bbox = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\BBox\BBox_large.shp"
weather_param = ['winddir', 'atmostemp']

filter_acis_station_data(csv_path, months, shp_bbox, weather_param)


Number of unique stations within the bounding box: 27
Filtered ACIS data saved at: D:\UCalgary_Lectures\GEOG_683\Data_workspace\BBox\filtered_acis_data_BBox_large_1.csv


In [30]:
import pandas as pd
from pathlib import Path
import numpy as np

def calculate_wind_direction_hour_counts_to_csv(input_csv_file, bin_size=15, cutoff_day=None):
    """
    Processes ACIS wind data from a CSV to calculate the number of hours the wind falls within specific direction bins 
    for each station and month, preserving hourly counts.

    Parameters:
    - input_csv_file: Path to the input CSV file.
    - bin_size: Wind direction bin size in degrees (default is 15°).
    - cutoff_day: Limits data to days 1 through cutoff_day per month (default is None for full month).
    """
    # Load CSV data
    df = pd.read_csv(input_csv_file)
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    df.dropna(subset=['datetime', 'winddir', 'station name'], inplace=True)
    
    # Apply day cutoff if specified
    if cutoff_day:
        df = df[df['datetime'].dt.day <= cutoff_day]
    
    # Extract year, month, and ensure wind direction is in degrees
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    
    # Bin wind direction data into specified intervals
    bins = np.arange(0, 360 + bin_size, bin_size)
    df['wind_dir_bin'] = pd.cut(df['winddir'], bins=bins, 
                                labels=[f"{int(b)}-{int(b + bin_size)}" for b in bins[:-1]], right=False)

    # Count occurrences within each bin by station, year, and month
    wind_direction_counts = df.groupby(['station name', 'year', 'month', 'wind_dir_bin']).size().unstack(fill_value=0)

    # Save output to CSV
    output_csv_file = Path(input_csv_file).with_name(f"acis_wind_direction_{bin_size}deg_{Path(input_csv_file).stem}.csv")
    wind_direction_counts.to_csv(output_csv_file)
    print(f"Wind direction hour counts saved to {output_csv_file}")

# Example usage
input_csv_file = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\BBox\filtered_acis_data_BBox_large_1.csv"
calculate_wind_direction_hour_counts_to_csv(input_csv_file)


Wind direction hour counts saved to D:\UCalgary_Lectures\GEOG_683\Data_workspace\BBox\acis_wind_direction_15deg_filtered_acis_data_BBox_large_1.csv


In [31]:
import pandas as pd
import re
from pathlib import Path

def process_acis_binned_data(input_csv, aggregation='yearly'):
    """
    Processes ACIS weather station data with pre-binned wind directions, counting hours within specific direction groups
    (210-279 and 280-209) and outputs a pivoted CSV file.

    Parameters:
    - input_csv: Path to the input ACIS CSV file containing pre-binned wind direction data.
    - aggregation: Aggregation level, either 'yearly' or '5-yearly'.
    """
    df = pd.read_csv(input_csv, low_memory=False)

    # Identify columns corresponding to each wind direction bin
    dir_210_279_bins = [col for col in df.columns if re.match(r'^(21[0-9]|2[0-7][0-9])-2[0-7][0-9]$', col)]
    dir_other_bins = [col for col in df.columns if re.match(r'^\d+-\d+$', col) and col not in dir_210_279_bins]

    # Sum counts for the two direction groups
    df['wind_dir_210_279'] = df[dir_210_279_bins].sum(axis=1)
    df['wind_dir_280_209'] = df[dir_other_bins].sum(axis=1)

    # Determine aggregation period
    if aggregation == '5-yearly':
        df['aggregation_period'] = (df['year'] // 5 * 5).astype(str) + '-' + (df['year'] // 5 * 5 + 4).astype(str)
        pivot_table = df.pivot_table(
            values=['wind_dir_210_279', 'wind_dir_280_209'],
            index=['station name', 'aggregation_period', 'month'],
            aggfunc='sum'
        ).reset_index()
    elif aggregation == 'yearly':
        df['aggregation_period'] = df['year'].astype(str)
        pivot_table = df.pivot_table(
            values=['wind_dir_210_279', 'wind_dir_280_209'],
            index=['station name', 'aggregation_period', 'month'],
            aggfunc='sum'
        ).reset_index()
    else:
        raise ValueError("Invalid aggregation level. Please specify either 'yearly' or '5-yearly'.")

    # Save the results to a new CSV file
    output_csv = Path(input_csv).with_name(f"{Path(input_csv).stem}_{aggregation}_pivoted.csv")
    pivot_table.to_csv(output_csv, index=False)
    print(f"Processed data has been saved to {output_csv}")

# Example usage
input_csv_path = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\BBox\acis_wind_direction_15deg_filtered_acis_data_BBox_large_1.csv"
process_acis_binned_data(input_csv_path, aggregation='yearly')


Processed data has been saved to D:\UCalgary_Lectures\GEOG_683\Data_workspace\BBox\acis_wind_direction_15deg_filtered_acis_data_BBox_large_1_yearly_pivoted.csv
